# Mapping Part 2 — Workbook

In this lesson, we're going to learn how to analyze and visualize geographic data.

*Note: You can explore this [workbook](https://mybinder.org/v2/gh/INFO1350/Intro-CA-SP21/master?urlpath=lab/tree/book/07-Mapping/01.5-Mapping-WORKBOOK.ipynb) in the cloud via Binder.*

# Making Interactive Maps

To create interactive maps, we're going to use the Python library [Folium](https://python-visualization.github.io/folium/). Folium is built on top of the popular JavaScript library [Leaflet](https://leafletjs.com/).

In [2]:
import pandas as pd

In [3]:
ithaca_df = pd.read_csv("confirmed-places.csv")
ithaca_df

,place,count
0,Mexico,4
1,Texas,3
2,Loomis,2
3,Paulina,2
4,France,2
5,Puerto Rico,2
6,North Broadway,2
7,Chicago,1
8,Milwaukee,1
9,Uptown,1


In [4]:
from geopy.geocoders import Nominatim

### Nominatim & OpenStreetMap

Nominatim (which means "name" in Latin) uses [OpenStreetMap data](https://www.openstreetmap.org/relation/174979) to match addresses with geopgraphic coordinates. Though we don't need an API key to use Nominatim, we do need to create a unique [application name](https://operations.osmfoundation.org/policies/nominatim/). 

Here we're initializing Nominatim as a variable called `geolocator`. Change the application name below to your own application name:

In [5]:
geolocator = Nominatim(user_agent="YOUR NAME's mapping app", timeout=2)

To geocode an address or location, we simply use the `.geocode()` function:

## Geocode with Pandas

To geocode every place in a Pandas DataFrame, we can use two options.

### Option 1
First, we can make a list of place dictionaries, and then turn that list into a DataFrame.

Fill in the correct `values` for the `keys` "address," "latitude," "longitude," and "importance" in the dictionary below.

If you need help, look back at how we were accessing this information above. To test your code, see what the `ithaca_df` DataFrame looks like by running the cell below this cell. 

In [7]:
# Empty list
locations = []

# Loop through every place in the columns "places"
for place in ithaca_df['place']:
    
    #Geolocate the place
    location = geolocator.geocode(place, exactly_one=True)
        
    # Append a dictionary to the list locations
    locations.append({'place': place,
                      'address': location.address,
                      'latitude': location.latitude,
                      'longitude': location.longitude,
                      'importance': location.raw['importance']})

In [8]:
ithaca_df = pd.DataFrame(locations)
ithaca_df

,place,address,latitude,longitude,importance
0,Mexico,México,22.500048,-100.000038,0.839924
1,Texas,"Texas, United States",31.816038,-99.512099,0.868052
2,Loomis,"Loomis, Placer County, California, 95650, Unit...",38.821289,-121.193004,0.530535
3,Paulina,"Paulina, gmina Kcynia, powiat nakielski, wojew...",53.092920,17.492263,0.429019
4,France,France,46.603354,1.888334,1.023326
5,Puerto Rico,"Puerto Rico, United States",18.221415,-66.413282,0.931797
6,North Broadway,"North Broadway, White Plains, Westchester Coun...",41.041942,-73.768170,0.300000
7,Chicago,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421,0.861530
8,Milwaukee,"Milwaukee, Milwaukee County, Wisconsin, United...",43.034993,-87.922497,0.685259
9,Uptown,"Uptown, Houston, Harris County, Texas, 77056, ...",29.745802,-95.464244,0.483582


## Option 2

Second, we can create a function, and then `.apply()` that function to the existing DataFrame. This code should run without alteration.

In [17]:
def find_location(row):
    
    place = row['place']
    location = geolocator.geocode(place)
    
    return location.address, location.latitude, location.longitude, location.raw['importance']

In [18]:
ithaca_df[['address', 'latitude', 'longitude', 'importance']] = ithaca_df.apply(find_location,
                                                                     axis="columns",
                                                                     result_type="expand")
ithaca_df

,place,address,latitude,longitude,importance
0,College Town Bagels,"Collegetown Bagels, 415, College Avenue, East ...",42.442254,-76.485473,0.350000
1,Ithaca Falls,"Ithaca Falls, Ithaca, Ithaca Town, Tompkins Co...",42.452821,-76.491637,0.480840
2,Moosewood Restaurant,"Moosewood Restaurant, 215, North Cayuga Street...",42.440603,-76.498632,0.475949
3,Cascadilla Gorge Trail,"Cascadilla Gorge Trail, University Hill, Ithac...",42.442968,-76.493513,0.375000
4,Goldwin-Smith Hall,"Goldwin Smith Hall, 232, Feeney Way, Ithaca, I...",42.449069,-76.483479,0.301000
5,Carriage House Cafe,"Carriage House Cafe, Stewart Avenue, Ithaca, I...",42.442009,-76.489704,0.301000
6,Olin Library,"Olin Library, Ho Plaza, Ithaca, Ithaca Town, T...",42.447823,-76.484281,0.201000
7,Purity Ice Cream,"Purity Ice Cream, 700, Cascadilla Street, Itha...",42.444465,-76.508855,0.301000
8,Buttermilk Falls State Park,"Buttermilk Falls State Park, Ithaca Town, Tomp...",42.416166,-76.516440,0.771981
9,Libe Slope,"Libe Slope, Ithaca, Ithaca Town, Tompkins Coun...",42.448371,-76.486638,0.400000


### Install Folium

In [ ]:
!pip install folium

### Import Python packages

In [1]:
import pandas as pd

In [10]:
import folium

### Base Map

First, we need to establish a base map. To do so, we're going to call `folium.Map()`and enter the general latitude/longitude coordinates of the Ithaca area at a particular zoom.

(To find latitude/longitude coordintes for a particular location, you can use Google Maps, [as described here](https://support.google.com/maps/answer/18539?co=GENIE.Platform%3DDesktop&hl=en).)

In [11]:
base_map = folium.Map(location=[41.908114613735904, -87.6231507038231], zoom_start=14)
base_map

What if we wanted to map a bunch of places in the Ithaca area?

# Geocoding with GeoPy

To mape these places, we need their latitude/longitude coordinates. So we're going to geocode them with the Python package [GeoPy](https://geopy.readthedocs.io/en/stable/#).

GeoPy makes it easier to use a range of third-party [geocoding API services](https://geopy.readthedocs.io/en/stable/#), such as Google, Bing, ArcGIS, and OpenStreetMap.

Though most of these services require an API key, Nominatim, which uses OpenStreetMap data, does not, which is why we're going to use it here.

### Install GeoPy

In [ ]:
!pip install geopy

### Import Nominatim

From GeoPy's list of possible geocoding services, we're going to import Nominatim:

### Add Markers From Pandas Data

`folium.Marker(location=[Latitude, Longitude], popup="Place Name").add_to(base_map)`


To add markers for every location in our Pandas dataframe, we can similarly use two options.

### Option 1

We can loop through the DataFrame with `.iterrows()`

In [12]:
for index, row in ithaca_df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['place']).add_to(base_map)

base_map

### Option 2

Create a function and apply it to the DataFrame

In [20]:
def create_map_markers(row, map_name):
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['place']).add_to(map_name)

In [21]:
ithaca_df.apply(create_map_markers, map_name=base_map, axis='columns')

base_map

### Save Map

In [22]:
base_map.save("Ithaca-map.html")